In [2]:
import re
from lizard import analyze_file

In [91]:
code = "public <U> boolean accept(Subscriber<? super U> subscriber) {\n        Object[] a = head;\n        final int c = capacity;\n        while (a != null) {\n            for (int i = 0; i < c; i++) {\n                Object o = a[i];\n                if (o == null) {\n                    break;\n                }\n\n                if (NotificationLite.acceptFull(o, subscriber)) {\n                    return true;\n                }\n            }\n            a = (Object[])a[c];\n        }\n        return false;\n    }"

In [111]:
code = "<?php\npublic function show($table)\n    {\n        $this->authorize('browse_database');\n\n        $additional_attributes = [];\n        $model_name = Voyager::model('DataType')->where('name', $table)->pluck('model_name')->first();\n        if (isset($model_name)) {\n            $model = app($model_name);\n            if (isset($model->additional_attributes)) {\n                foreach ($model->additional_attributes as $attribute) {\n                    $additional_attributes[$attribute] = [];\n                }\n            }\n        }\n\n        return response()->json(collect(SchemaManager::describeTable($table))->merge($additional_attributes));\n    }\n?>"

In [113]:
i = analyze_file.analyze_source_code(filename='abc.php', code=code)

In [114]:
i.__dict__

{'filename': 'abc.php',
 'nloc': 15,
 'function_list': [<lizard.FunctionInfo at 0x7f459c553730>],
 'token_count': 113}

In [115]:
for func in i.function_list:
    print(func.long_name)

show ( $table )


In [38]:
def extract_function_params(function_code, comment, file_name,):
    metadata = {}
    report = analyze_file.analyze_source_code(file_name, function_code)
    # print(function_code)
    
    params = report.function_list[0].full_parameters
    # print(params)
    params_dict = {}
    # params_name = [str(x).split(' ')[1] for x in params] 
    for each in params:
        line = str(each).split(' ')
        params_dict[line[-1]] = {'type': line[0]}
        params_dict[line[-1]] = {'comment': False}
        
    # print(params)
        
    preprocessed = re.split(r"(@+[\w]*)", comment)

    splited_comment = []
    line = ""
    for sub_str in preprocessed:
        if re.search(r"(@+[\w]*)", sub_str):
            splited_comment.append(line)
            line = re.search(r"(@+[\w]*)", sub_str).group()
        else: 
            line += sub_str
    splited_comment.append(line)
    
    # print(splited_comment)
    
    for line in splited_comment:
        if re.search(r"(@+[\w]*)", line):
            break_line = line.split(' ')
            # print(break_line[0])
            if break_line[0] == '@param':
                params_dict[break_line[1]] = ' '.join(break_line[2:])

            else:
                params_dict[break_line[0]] = ' '.join(break_line[1:])
        else:
            metadata['processed_docstring'] = line
    
    metadata['docstring_params'] = params_dict
    return metadata
            
# print(code)
a_dict = extract_function_params(code, comment, 'AppendOnlyLinkedArrayList.java')
print(a_dict)

{'processed_docstring': 'Interprets the contents as NotificationLite objects and calls\nthe appropriate Subscriber method.\n\n', 'docstring_params': {'subscriber': 'the subscriber to emit the events to\n', '<U>': 'the target type\n', '@return': 'true if a terminal event has been reached'}}


In [40]:
DOCSTRING_REGEX_TOKENIZER = re.compile(r"[^\s,'\"`.():\[\]=*;>{\}+-/\\]+|\\+|\.+|\(\)|{\}|\[\]|\(+|\)+|:+|\[+|\]+|{+|\}+|=+|\*+|;+|>+|\++|-+|/+")


def tokenize_docstring(docstring: str):
    return [t for t in DOCSTRING_REGEX_TOKENIZER.findall(docstring) if t is not None and len(t) > 0]

In [41]:
print(tokenize_docstring(a_dict['processed_docstring']))

['Interprets', 'the', 'contents', 'as', 'NotificationLite', 'objects', 'and', 'calls', 'the', 'appropriate', 'Subscriber', 'method', '.']


In [48]:
a_dict

{'processed_docstring': 'Interprets the contents as NotificationLite objects and calls\nthe appropriate Subscriber method.\n\n',
 'docstring_params': {'subscriber': 'the subscriber to emit the events to\n',
  '<U>': 'the target type\n',
  '@return': 'true if a terminal event has been reached'}}

In [125]:
preprocessed = re.findall(r"@+[\w]*", comment)

In [126]:
preprocessed

['@param', '@param', '@return']

In [49]:
java_param_docstring = [
    '@author',
    '@code',
    '@exception',
    '@return',
    '@see',
    '@throws',
    '@value',
    '@version',
    '@param',
]

In [50]:
java_param_docstring[2:]

['@exception', '@return', '@see', '@throws', '@value', '@version', '@param']

In [69]:
code = "function getCounts(langs = []) {\n    return {\n        langs: langs.length,\n        modelLangs: langs.filter(({ models }) => models && !!models.length).length,\n        models: langs.map(({ models }) => (models ? models.length : 0)).reduce((a, b) => a + b, 0),\n    }\n}"